# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 21, 7, 0

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            print(dt.hour, dt.minute)
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = True

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [20, 25, 30, 35]},
              'STOCH': {'fastk_period': [3, 5, 7, 9, 11], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [2, 3, 5, 7, 9], 'low_bound': [10, 15, 20, 25]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 4800


  0%|                                                                                                                                                                                                 | 0/4800 [00:00<?, ?it/s]


Load the datasets...
Binance
ByBit
BinanceFutures
ByBitPerpetual


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4800/4800 [2:44:57<00:00,  2.06s/it]


# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  3                  10                            75.759735   
                                                                   5                  10                            83.794091   
               35            11                 3                  9                  20                            66.518698   
               25            7                  3                  5                  10                            88.219697   
12             20            7                  3                  5                  10                            86.237538   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.443447   
14             30            5                  4                  5                  10                            72.603182   
12             20            7                  4                  5                  10                            59.343523   
16             25            9                  3                  5                  10                            73.011894   
                             7                  3                  7                  10                            56.439394   
12             20            7                  3                  7                  10                            65.530303   
16             25            7                  4                  5                  10                            54.286970   
12             20            5                  3                  7                  10                            58.333333   
16             25            5                  3                  7                  10                            58.333333   
12             20            9                  3                  5                  10                            73.201098   
               30            5                  4                  5                  10                            69.466932   
16             25            7                  2                  5                  10                            77.102348   
               35            5                  4                  5                  10                            69.892652   
14             20            5                  5                  5                  10                            75.757576   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  3                  10                         0.843144   
                                                                   5                  10                         1.436705   
               35            11                 3                  9                  20                         0.455156   
               25            7                  3                  5                  10                         1.885038   
12             20            7                  3                  5                  10                         1.803295   
               30            11                 3                  9                  25                         0.418698   
               25            5                  4                  5                  10                         1.366742   
14             30            5                  4                  5                  10                         0.644697   
12            

# Save new config data to config file

In [1]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [35]},
                  'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                            'slowd_period': [3], 'low_bound': [15]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

In [16]:
stat[(stat['RSI_timeperiod'] == 14) & (stat['RSI_low_bound'] == 35) & (stat['STOCH_fastk_period'] == 9) & 
          (stat['STOCH_slowk_period'] == 3) & (stat['STOCH_slowd_period'] == 7) & 
          (stat['STOCH_low_bound'] == 25)]

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,...,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
2015,STOCH_RSI,14,35,9,3,7,25,84.80,71.35,65.50,...,0.07,0.00,-0.20,-0.30,-0.17,171,51.608750,0.001250,-3144.903750,0.213750
2015,STOCH_RSI,14,35,9,3,7,25,84.31,66.67,64.22,...,0.34,0.29,0.24,0.17,0.18,204,54.575000,0.109167,-3146.700000,22.270000
2459,STOCH_RSI,14,35,9,3,7,25,77.91,59.30,54.65,...,0.10,0.08,0.00,0.12,0.10,86,54.360833,0.088750,-1344.968333,7.632500
3075,STOCH_RSI,14,35,9,3,7,25,96.97,81.82,69.70,...,-0.15,-0.18,-0.26,-0.10,-0.50,33,51.892083,0.033750,-597.561250,1.113750
3075,STOCH_RSI,14,35,9,3,7,25,91.67,75.00,72.22,...,0.01,0.00,0.15,0.24,0.24,36,59.028750,0.201250,-394.965000,7.245000
3075,STOCH_RSI,14,35,9,3,7,25,81.03,60.34,53.45,...,0.50,0.38,0.46,0.33,0.42,58,59.912500,0.246667,-585.075000,14.306667
3075,STOCH_RSI,14,35,9,3,7,25,87.50,66.07,67.86,...,0.77,0.65,0.44,0.07,0.47,56,61.161667,0.388750,-494.946667,21.770000
3075,STOCH_RSI,14,35,9,3,7,25,92.68,75.61,63.41,...,-0.27,-0.21,-0.27,-0.63,-0.63,41,52.437083,-0.013333,-720.079583,-0.546667
2015,STOCH_RSI,14,35,9,3,7,25,83.25,66.01,62.07,...,0.05,0.00,-0.18,-0.27,-0.09,203,49.465417,-0.037500,-4168.520417,-7.612500
2459,STOCH_RSI,14,35,9,3,7,25,85.19,75.00,72.22,...,0.73,0.71,0.93,1.06,1.00,108,70.447500,0.486250,48.330000,52.515000


In [11]:
total_stat.loc[(14, 35, 9, 3, 7, 25)]

pct_right_forecast_avg      57.038258
pct_price_diff_avg           0.158068
forecast_rank            -1413.654773
price_rank                  13.657273
forecasts_num             1130.000000
Name: (14, 35, 9, 3, 7, 25), dtype: float64